<h1> v3, premade puzzles + deck manipulation </h1>

In this version, we generate a bank of puzzles on a difficulty scale between 0 and 1.  
In-game events and the players' skill level determine the level of difficulty of premade puzzle to serve the player.  
Contributors include:
- Win streak / loss streak
- Average word length
- ...  
<hr>


We also create a toggleable mode that controls the deck reveal:  
- Guarantee Build Mode: Guarantees to either create a word from the board + deck, or to extend the length of the current word in the word builder space.  
- Normal: Random letter reveal
- Gaurantee Junk Mode: Guarantees to randomly reveal a letter from a list of letters that do NOT help with the current word / board.  Useful for incentivizing player to use power-ups and in-app purchases.
<hr>
<b> Logic for puzzle creation:   </b><br>
Each tile has a connectivity to other tiles on the board.  For example, those in the first row are 100% connected, as they are guaranteed to be revealed at the same time and can be played together.  Tiles in the second row are not guaranteed to all be revealed at the same time, but are still more likely to be available together compared to tiles in the 4th row for instance.  We create a connectivity coefficient (pointwise mutual information (PMI)) for each tile and the other tiles on the board.  Then we randomly select tiles, row by row, and populate a letter that generates the desired possibility of valid words being created.  When difficulty is high, we want to populate less word possibilities between highly connected letters, and the inverse when difficulty is low.  This process should lead to puzzles on the difficulty range (0,1).

In [1]:
# wordscapes_depth.py (single-mode + sinusoidal epsilon + charts)
# Python 3.7-compatible
import random, string, math, tkinter as tk
from tkinter import messagebox, ttk

# ----------------------------
# Config & Helpers
# ----------------------------
BUILTIN_WORDS = {
    "an","at","ate","ant","art","are","ear","earn","lean","learn",
    "ran","tan","tear","rate","rent","ten","net","tar","rat","tea",
    "late","near","lane","rant","enter","tree","treat",
    "cat","car","care","cart","crate","star","stare","stone","note","tone"
}

def load_words():
    for fname in ("words_alpha.txt","words_small.txt"):
        try:
            with open(fname, "r", encoding="utf-8") as f:
                words = {w.strip().lower() for w in f if w.strip()}
                return {w for w in words if w.isalpha() and 2 <= len(w) <= 6}
        except Exception:
            pass
    return BUILTIN_WORDS

LEXICON = load_words()

BASE_LETTER_WEIGHTS = {
    'E': 12, 'A': 9, 'R': 6, 'I': 7, 'O': 7, 'T': 9, 'N': 6, 'S': 6, 'L': 4,
    'C': 3, 'U': 3, 'D': 4, 'P': 2, 'M': 2, 'H': 6, 'G': 2, 'B': 2, 'F': 2,
    'Y': 2, 'W': 2, 'K': 1, 'V': 1, 'X': 1, 'Z': 1, 'J': 1, 'Q': 1
}

def sample_letter():
    letters, wts = zip(*BASE_LETTER_WEIGHTS.items())
    return random.choices(letters, weights=wts, k=1)[0]

def score_word(w): L=len(w); return 0 if L<2 else L*L

# ----------------------------
# Board with Depth Layers
# ----------------------------
class Tile:
    def __init__(self, tid, layer, col):
        self.id, self.layer, self.col = tid, layer, col
        self.revealed = self.cleared = False
        self.letter = None
        self.blocked_by = set()

# --- Deck modes ---
class DeckMode:
    NORMAL = "normal"
    GUARANTEE_BUILD = "build"
    GUARANTEE_JUNK = "junk"
        
class DeckCard:
    def __init__(self, letter):
        self.letter = letter
        self.selected = False


class Deck:
    """
    Deck with toggleable draw modes:
      - NORMAL: random letter (pre-populated or sampled)
      - GUARANTEE_BUILD: pick a letter that increases available words or extends the current prefix
      - GUARANTEE_JUNK: pick a letter that does NOT help (if possible)
    """
    def __init__(self, size=10, mode=DeckMode.NORMAL):
        self.cards = [DeckCard(sample_letter()) for _ in range(size)]
        self.pos = 0
        self.mode = mode
        self._junk_bag = []  # <- new: draw-without-replacement pool for junk

    # -- basic API stays the same --
    def remaining(self): 
        return max(0, len(self.cards) - self.pos)

    def can_draw(self): 
        return self.pos < len(self.cards)

    def set_mode(self, mode: str):
        self.mode = mode
        # reset bag when mode changes to avoid stale state
        self._junk_bag = []

    def draw_one(self, game=None):
        """
        Draw one card. If a GameUI `game` is provided and mode != NORMAL,
        the card's letter is chosen based on current board/stack state.
        """
        if not self.can_draw():
            return None

        card = self.cards[self.pos]
        self.pos += 1

        # If no game context or NORMAL mode, keep the pre-populated random letter.
        if game is None or self.mode == DeckMode.NORMAL:
            return card

        try:
            if self.mode == DeckMode.GUARANTEE_BUILD:
                letter = self._choose_build_letter(game)
            elif self.mode == DeckMode.GUARANTEE_JUNK:
                letter = self._choose_junk_letter(game)
            else:
                letter = sample_letter()
        except Exception as e:
            # Safety fallback if anything unexpected happens
            if hasattr(game, "log"):
                game.log(f"[DECK] Mode '{self.mode}' failed ({e}); using random.")
            letter = sample_letter()

        card.letter = letter
        return card

    # ---------- internals ----------
    def _candidate_deltas(self, game):
        """
        For each letter a–z, compute:
          - delta: change in number of formable words next round if we add that letter
          - extends: whether it extends the current selected prefix towards some lexicon word
        Uses GameUI helpers you've already implemented.
        """
        # State & helpers from your GameUI
        board_counts, stack_counts, stack_seq = game._next_round_counts()
        base_counts = game._add_counts(board_counts, stack_counts)
        before = game._exact_count_next_round(base_counts, stack_seq, extra_letter=None)

        # Current selected prefix
        try:
            prefix = (game.current_word() or "").lower()
        except Exception:
            prefix = ""

        # Use the precomputed list in GameUI if present; otherwise fallback to global LEXICON
        lex_words = getattr(game, "_lex_words", list(LEXICON))

        # Pre-index: does any lexicon word start with given prefix?
        def extends_prefix(ch):
            if not prefix:
                # "extending" an empty prefix just means there exists any word starting with ch
                pc = ch
            else:
                pc = prefix + ch
            # Quick early exit by length
            # Filter only if the prefix length <= max word length you're using (<=6 in your code)
            plen = len(pc)
            for w in lex_words:
                if len(w) >= plen and w.startswith(pc):
                    return True
            return False

        out = []
        for ch in string.ascii_lowercase:
            after = game._exact_count_next_round(base_counts, stack_seq, extra_letter=ch)
            delta = after - before
            out.append((ch, delta, extends_prefix(ch)))
        return out

    def _choose_build_letter(self, game):
        """
        Prefer letters that increase available words (delta>0),
        otherwise letters that extend the current prefix.
        Tie-break toward more positive delta (even if 0) and common letters.
        """
        cands = self._candidate_deltas(game)

        # Primary: delta>0 (creates new opportunities)
        positives = [(ch, d, ext) for (ch, d, ext) in cands if d > 0]
        if positives:
            # weight by delta (stronger gain more likely) with a small base weight
            weights = []
            for ch, d, _ in positives:
                basew = 0.5 + (BASE_LETTER_WEIGHTS.get(ch.upper(), 1))
                weights.append(max(1e-6, basew + 2.0 * d))
            return self._weighted_choice([ch for ch,_,_ in positives], weights).upper()

        # Secondary: no increase available; extend the user's current prefix if possible
        extenders = [(ch, d) for (ch, d, ext) in cands if ext]
        if extenders:
            # prefer higher delta (could be zero) and common letters
            extenders.sort(key=lambda x: (x[1], BASE_LETTER_WEIGHTS.get(x[0].upper(), 0)), reverse=True)
            return extenders[0][0].upper()

        # Fallback: nothing helps; pick a random consonant-ish letter to avoid too many vowels
        consonants = [ch for ch in string.ascii_uppercase if ch not in "AEIOU"]
        return random.choice(consonants)

    def _choose_junk_letter(self, game):
        """
        Build a pool of 'junk' letters (delta <= 0 and doesn't extend prefix),
        then pick uniformly at random, drawing without replacement until the pool changes
        or is exhausted. This reduces repeated junk letters across consecutive draws.
        """
        cands = self._candidate_deltas(game)

        # Unique junk letters (lowercase)
        junk_letters = sorted({ch for (ch, d, ext) in cands if d <= 0 and not ext})

        if junk_letters:
            # Maintain a draw-without-replacement bag aligned to the *current* junk set
            current_set = set(junk_letters)
            # Keep only still-valid letters in the existing bag
            self._junk_bag = [ch for ch in self._junk_bag if ch in current_set]

            # If bag empty, (re)seed it from current pool and shuffle
            if not self._junk_bag:
                self._junk_bag = list(junk_letters)
                random.shuffle(self._junk_bag)

            pick = self._junk_bag.pop()  # draw without replacement
            return pick.upper()

        # No strict junk exists right now → pick the least helpful overall
        def badness(t):
            ch, d, ext = t
            # lower is "worse": prefer smallest delta and avoid prefix extenders
            return (d + (1 if ext else 0))

        worst = min(cands, key=badness)
        if hasattr(game, "log"):
            game.log("[DECK] No strict junk available; revealing least-helpful letter.")
        return worst[0].upper()
    
    @staticmethod
    def _weighted_choice(items, weights):
        """items: list, weights: list of positive numbers"""
        total = sum(weights)
        r = random.random() * total
        acc = 0.0
        for it, w in zip(items, weights):
            acc += w
            if acc >= r:
                return it
        return items[-1]
        
class DepthBoard:
    def __init__(self, layer_counts=(6,8,10,12)):
        self.layer_counts = list(layer_counts)
        self.tiles=[]; tid=0
        for L,count in enumerate(self.layer_counts):
            for c in range(count):
                self.tiles.append(Tile(tid,L,c)); tid+=1
        self.index={(t.layer,t.col):t.id for t in self.tiles}
        self._wire_blockers()
        self.pick_letter_fn=None
        for t in self.tiles:
            if t.layer==0: self._reveal_assign_letter(t.id)
        self.soft_cleared=set()

    def _wire_blockers(self):
        for L in range(1,len(self.layer_counts)):
            up_n, lo_n = self.layer_counts[L-1], self.layer_counts[L]
            for lc in range(lo_n):
                if up_n==0: continue
                u_pos = (lc+0.5)*up_n/lo_n
                u_left = max(0, min(up_n-1, int(u_pos)))
                cands={u_left}
                if u_pos-u_left>0.15 and u_left+1<up_n: cands.add(u_left+1)
                lower_id=self.index[(L,lc)]
                for uc in cands:
                    upper_id=self.index[(L-1,uc)]
                    self.tiles[lower_id].blocked_by.add(upper_id)

    def _reveal_assign_letter(self, tid):
        t = self.tiles[tid]
        if t.cleared:
            return
        # Letter is now predetermined (by PuzzleGenerator) before the game starts.
        # If for any reason it's missing, keep a safe fallback to sampling:
        if t.letter is None:
            t.letter = sample_letter()
        t.revealed = True

    def set_soft_clear(self, tid, active=True):
        (self.soft_cleared.add if active else self.soft_cleared.discard)(tid)

    def clear_selected(self, tids):
        for tid in tids:
            t=self.tiles[tid]; t.cleared=True; t.revealed=False; self.soft_cleared.discard(tid)

    def _blocker_present(self, tid):
        t=self.tiles[tid]; return (not t.cleared) and (tid not in self.soft_cleared)

    def _can_reveal(self, tid):
        t=self.tiles[tid]
        return (not t.cleared) and all(not self._blocker_present(b) for b in t.blocked_by)

    def recompute_visibility(self):
        changed=True
        while changed:
            changed=False
            for t in self.tiles:
                if t.cleared: continue
                want = self._can_reveal(t.id)
                if want and not t.revealed: self._reveal_assign_letter(t.id); changed=True
                elif (not want) and t.revealed: t.revealed=False

# ----------------------------
# Deck & Revealed Stack
# ----------------------------
class DeckCard:
    def __init__(self, letter): self.letter, self.selected = letter, False

# class Deck:
#     def __init__(self, size=10):
#         self.cards=[DeckCard(sample_letter()) for _ in range(size)]; self.pos=0
#     def remaining(self): return max(0,len(self.cards)-self.pos)
#     def can_draw(self): return self.pos<len(self.cards)
#     def draw_one(self):
#         if not self.can_draw(): return None
#         card=self.cards[self.pos]; self.pos+=1; return card

# ----------------------------
# GUI
# ----------------------------
class GameUI:
    def __init__(self, root):
        self.root=root; root.title("Wordscapes Solitaire – Preloaded Puzzles + Deck Manipulation")

        # Stats
        self.stats=dict(games=0,wins=0,losses=0,words=0,total_len=0,total_score=0,total_game_score=0)
        self.reveal_events = 0
        self.total_new_words_from_flips = 0

        # Difficulty parameter (drives revealers). We'll *update* it sinusoidally after each tile reveal.
        self.diff_var = tk.DoubleVar(value=0.5)

        # Sinusoid controls & histories
        self.eps_period = 30          # tile reveals per sine cycle
        self.tile_tick = 0            # counts *tile* reveals (ε updates only here)
        self.reveal_tick = 0          # counts *all* reveals (tile + deck) for shared x-axis
        self.epsilon_history = [float(self.diff_var.get())]  # seed with initial ε
        self.newflip_raw_history = []  # Δ values per reveal (tile+deck)
        self.eps_markers = []         # (index_in_history, is_win_bool)

        # Game state
        self.game_over=False; self.score_total=0; self.selected=[]

        # Top
        top=tk.Frame(root); top.pack(pady=6, fill="x")
        self.score_var=tk.StringVar(value="Score: 0"); tk.Label(top,textvariable=self.score_var,font=("Arial",12,"bold")).pack(side="left",padx=8)
        self.preview_var=tk.StringVar(value="Word: —  (score: 0)"); tk.Label(top,textvariable=self.preview_var,font=("Arial",12)).pack(side="left",padx=8)
        self.game_status=tk.StringVar(value="Game: In progress"); tk.Label(top,textvariable=self.game_status,font=("Arial",11,"italic")).pack(side="right",padx=8)

        # Middle: board + stats
        mid=tk.Frame(root); mid.pack(padx=10,pady=8,fill="both",expand=True)
        self.board_frame=tk.Frame(mid); self.board_frame.pack(side="left",padx=(0,12))
        ttk.Separator(mid,orient="vertical").pack(side="left",fill="y",padx=6)

        stats_panel=tk.Frame(mid,relief="groove",bd=2); stats_panel.pack(side="right",fill="y")
        tk.Label(stats_panel,text="Player Stats",font=("Arial",12,"bold")).pack(anchor="w",padx=8,pady=(6,4))
        self.stats_winloss=tk.StringVar(value="Wins-Losses: 0-0 (0.0%)")
        self.stats_avg_len=tk.StringVar(value="Avg Word Length: 0.0")
        self.stats_avg_score=tk.StringVar(value="Avg Word Score: 0.0")
        self.stats_avg_game=tk.StringVar(value="Avg Game Score: 0.0")
        self.stats_avg_flip=tk.StringVar(value="Avg +New/Flip: 0.00")
        for var in (self.stats_winloss,self.stats_avg_len,self.stats_avg_score,self.stats_avg_game,self.stats_avg_flip):
            tk.Label(stats_panel,textvariable=var,anchor="w").pack(fill="x",padx=8)

        # ε live value
        ttk.Separator(stats_panel, orient="horizontal").pack(fill="x", padx=8, pady=(6,4))
        self.eps_value_var = tk.StringVar(value="ε: 0.50 (sinusoidal)")
        tk.Label(stats_panel, textvariable=self.eps_value_var, anchor="w", font=("Arial",10,"bold")).pack(fill="x", padx=8)

        # Charts
        tk.Label(stats_panel, text="Epsilon over time").pack(anchor="w", padx=8, pady=(6,0))
        self.eps_canvas_w,self.eps_canvas_h=260,100
        self.eps_canvas=tk.Canvas(stats_panel,width=self.eps_canvas_w,height=self.eps_canvas_h,bg="#fafafa",
                                  highlightthickness=1,highlightbackground="#ccc")
        self.eps_canvas.pack(padx=8, pady=(0,8))

        tk.Label(stats_panel, text="New/Flip Δ over time").pack(anchor="w", padx=8, pady=(0,0))
        self.nf_canvas_w,self.nf_canvas_h=260,100
        self.nf_canvas=tk.Canvas(stats_panel,width=self.nf_canvas_w,height=self.nf_canvas_h,bg="#fafafa",
                                 highlightthickness=1,highlightbackground="#ccc")
        self.nf_canvas.pack(padx=8, pady=(0,8))
        
        # New/Flip smoothing control
        smooth_row = tk.Frame(stats_panel)
        smooth_row.pack(anchor="w", padx=8, pady=(0,8))
        tk.Label(smooth_row, text="New/Flip smoothing (k):").pack(side="left")

        self.nf_smooth_window = tk.IntVar(value=10)  # default k=10
        self.nf_smooth_spin = tk.Spinbox(
            smooth_row, from_=1, to=100, width=4,
            textvariable=self.nf_smooth_window,
            command=self.update_nf_smoothing
        )
        self.nf_smooth_spin.pack(side="left", padx=(6,0))
        self.nf_smooth_spin.bind("<Return>", lambda e: self.update_nf_smoothing())
        self.nf_smooth_spin.bind("<FocusOut>", lambda e: self.update_nf_smoothing())


        # Controls
        controls=tk.Frame(root); controls.pack(pady=6)
        self.confirm_btn=tk.Button(controls,text="Confirm Word",command=self.on_confirm,state="disabled"); self.confirm_btn.grid(row=0,column=0,padx=6)
        tk.Button(controls,text="Clear Selection",command=self.clear_selection).grid(row=0,column=1,padx=6)
        tk.Button(controls,text="Backspace (remove last)",command=self.backspace_last).grid(row=0,column=2,padx=6)
        tk.Button(controls,text="Suggest Longest (Board)",command=self.ui_suggest_longest_board).grid(row=0,column=3,padx=6)
        tk.Label(controls,text="Max Deck Cards to Use:").grid(row=0,column=4,padx=(16,2))
        self.stack_spin=tk.Spinbox(controls,from_=0,to=10,width=3); self.stack_spin.delete(0,"end"); self.stack_spin.insert(0,"2")
        self.stack_spin.grid(row=0,column=5,padx=2)
        tk.Button(controls,text="Suggest Longest (Board+Deck)",command=self.ui_suggest_longest_board_stack).grid(row=0,column=6,padx=6)

        # Stock / Revealed stack
        stock=tk.Frame(root); stock.pack(pady=8)
        self.draw_btn=tk.Button(stock,text="Draw from Deck",command=self.draw_from_deck); self.draw_btn.grid(row=0,column=0,padx=6)
        self.deck_var=tk.StringVar(value="Deck remaining: 0"); tk.Label(stock,textvariable=self.deck_var,font=("Arial",11)).grid(row=0,column=1,padx=6)
        self.stack_frame=tk.Frame(root); self.stack_frame.pack(pady=4); self.stack_btns=[]

        # Debug Log
        log_frame=tk.Frame(root); log_frame.pack(fill="both",expand=False,padx=10,pady=(0,10))
        tk.Label(log_frame,text="Debug Log:").pack(anchor="w")
        self.log_text=tk.Text(log_frame,height=6,width=88,state="disabled")
        self.log_text.pack(side="left",fill="both",expand=True)
        sb=tk.Scrollbar(log_frame,command=self.log_text.yview); sb.pack(side="right",fill="y")
        self.log_text.config(yscrollcommand=sb.set)

        # Footer
        footer=tk.Frame(root); footer.pack(pady=(0,10))
        self.new_game_btn=tk.Button(footer,text="NEW GAME",command=self.new_game,state="disabled"); self.new_game_btn.grid(row=0,column=0,padx=8)
        self.run_bot_btn=tk.Button(footer,text="Run Bot (10 games)",command=self.run_bot_10); self.run_bot_btn.grid(row=0,column=1,padx=8)

        self.tile_buttons={}
        self._precompute_lexicon_counts()
        # self.start_new_board()
        
        mode_row = tk.Frame(root); mode_row.pack(pady=(2,0))
        tk.Label(mode_row, text="Deck Mode:").pack(side="left", padx=(0,6))
        self.deck_mode_var = tk.StringVar(value=DeckMode.NORMAL)
        mode_box = ttk.Combobox(mode_row, width=16, textvariable=self.deck_mode_var, state="readonly",
                                values=[DeckMode.NORMAL, DeckMode.GUARANTEE_BUILD, DeckMode.GUARANTEE_JUNK])
        mode_box.pack(side="left")
        def _on_mode_change(_=None):
            self.deck.set_mode(self.deck_mode_var.get())
        mode_box.bind("<<ComboboxSelected>>", _on_mode_change)               
        self.deck = Deck(size=10, mode=self.deck_mode_var.get())
        
        # Difficulty selector (for puzzle loading/generation)
        diff_row = tk.Frame(root)
        diff_row.pack(pady=(6, 0))
        tk.Label(diff_row, text="Puzzle Difficulty:").pack(side="left", padx=(0,6))
        # common presets; you can freely type any 0<d<1 too
        self.diff_var_game = tk.DoubleVar(value=0.50)
        self.diff_menu = ttk.Combobox(
            diff_row, width=8, state="readonly",
            values=["0.50","0.25","0.50","0.75","0.90"]
        )
        self.diff_menu.set("0.50")
        self.diff_menu.pack(side="left")        
        
        self.new_game_from_file(f"puzzles/{int(self.diff_var_game.get()*100)}.json")
        
        root.bind("<BackSpace>", self._kb_backspace)

    # ---------- Generic utilities ----------
    def log(self,msg):
        self.log_text.config(state="normal"); self.log_text.insert("end",msg+"\n"); self.log_text.see("end"); self.log_text.config(state="disabled")

    def _stack_top_selected_run(self):
        top=0; i=len(self.stack)-1
        while i>=0 and self.stack[i].selected: top+=1; i-=1
        return top, len(self.stack)-1-top  # (selected_run, next_usable_idx)

    # ---------- Game lifecycle ----------
    def start_new_board(self):
        self.board=DepthBoard(layer_counts=(6,8,10,12))
        self.board.pick_letter_fn=self.pick_letter_for_reveal
        self.deck=Deck(size=10)
        self.stack=[]; self.selected=[]
        self.score_total=0; self.score_var.set("Score: 0")
        self.game_status.set("Game: In progress"); self.game_over=False
        self.draw_btn.config(state="normal"); self.new_game_btn.config(state="disabled")

        for w in list(self.board_frame.grid_slaves()): w.destroy()
        self.tile_buttons={}; self._build_layout()

        # NOTE: We intentionally do NOT reset:
        # - self.epsilon_history
        # - self.newflip_raw_history
        # - self.eps_markers
        # - self.reveal_tick / reveal counters
        # This keeps the graphs continuous across games.

        self.board.recompute_visibility(); self._refresh_all()
        self.deck_var.set(f"Deck remaining: {self.deck.remaining()}")
        self.update_stats_panel()
        self._draw_epsilon_chart()
        self._draw_newflip_chart()        

    def end_game(self, result):
        if self.game_over: return
        self.game_over=True; self.new_game_btn.config(state="normal")

        is_win = (result=="WIN")
        if is_win:
            self.stats["wins"]=self.stats.get("wins",0)+1
            self.game_status.set("Game: ✅ You win!")
            self.log("== GAME OVER: WIN ==")
        else:
            self.stats["losses"]=self.stats.get("losses",0)+1
            self.game_status.set("Game: ❌ No moves left — you lose.")
            self.log("== GAME OVER: LOSS ==")

        self.stats["games"]=self.stats.get("wins",0)+self.stats.get("losses",0)
        self.stats["total_game_score"]=self.stats.get("total_game_score",0)+self.score_total
        self.update_stats_panel()

        # Add marker at the most recent ε sample
        if len(self.epsilon_history) == 0:
            # ensure there is at least one point
            self.epsilon_history.append(float(self.diff_var.get()))
        marker_index = len(self.epsilon_history) - 1
        self.eps_markers.append((marker_index, is_win))
        self._draw_epsilon_chart()   # redraw with the new marker

        # Disable action buttons, but DO NOT clear the charts/histories
        self.confirm_btn.config(state="disabled")
        self.draw_btn.config(state="disabled")


    def new_game(self): self.log("== NEW GAME =="); self.start_new_board()

    # ---------- Stats & charts ----------
    def _classify_game_score(self,s): return "bad" if s<150 else ("good" if s<=225 else "great")

    def update_stats_panel(self):
        wins=self.stats.get("wins",0); losses=self.stats.get("losses",0)
        games=max(1,wins+losses)
        self.stats_winloss.set(f"Wins-Losses: {wins}-{losses} ({100.0*wins/games:.1f}%)")
        words=self.stats["words"]
        avg_len=(self.stats["total_len"]/words) if words else 0.0
        avg_score=(self.stats["total_score"]/words) if words else 0.0
        self.stats_avg_len.set(f"Avg Word Length: {avg_len:.1f}")
        self.stats_avg_score.set(f"Avg Word Score: {avg_score:.1f}")
        gp=self.stats["games"]
        avg_game=(self.stats["total_game_score"]/gp) if gp else 0.0
        self.stats_avg_game.set(f"Avg Game Score: {avg_game:.1f}")
        avg_new=(self.total_new_words_from_flips/self.reveal_events) if self.reveal_events else 0.0
        self.stats_avg_flip.set(f"Avg +New/Flip: {avg_new:.2f}")
        self.eps_value_var.set(f"ε: {float(self.diff_var.get()):.2f} (sinusoidal)")

    def _draw_epsilon_chart(self):
        c=self.eps_canvas; c.delete("all"); w,h=self.eps_canvas_w,self.eps_canvas_h
        c.create_rectangle(1,1,w-1,h-1,outline="#ccc")
        # guide lines at 0.0 and 0.50
        for thr,col in ((0.0,"#cde"),(0.50,"#edc")):
            y=10+(1.0-thr)*(h-20)
            c.create_line(2,y,w-2,y,fill=col,dash=(2,2))
            c.create_text(w-6,y-8,text=f"{thr:.2f}",anchor="e",fill="#888",font=("Arial",8))

        n = max(len(self.epsilon_history), len(self.newflip_raw_history), 1)
        if n <= 1:
            return
        lp,rp=8,8
        xs = n - 1
        step=(w-lp-rp)/xs if xs else (w-lp-rp)

        # ε polyline
        pts=[]
        for i in range(len(self.epsilon_history)):
            eps = self.epsilon_history[i]
            x=lp+i*step; y=10+(1.0-eps)*(h-20)
            pts += [x,y]
        if len(pts)>=4: c.create_line(*pts,fill="#2b6",width=2)

        # win/loss markers
        for idx,is_win in getattr(self, "eps_markers", []):
            if not (0 <= idx < n): 
                continue
            x=lp+idx*step
            eps = self.epsilon_history[idx] if idx < len(self.epsilon_history) else self.epsilon_history[-1]
            y=10+(1.0-eps)*(h-20)
            c.create_text(x, y-6, text="✓" if is_win else "X",
                          fill="#1a7f1a" if is_win else "#c02020",
                          font=("Arial", 10, "bold"))
            
    def update_nf_smoothing(self):
        """Validate smoothing window and redraw the New/Flip chart."""
        try:
            k = int(self.nf_smooth_window.get())
        except Exception:
            k = 10
        if k < 1: k = 1
        if k > 100: k = 100
        self.nf_smooth_window.set(k)
        self._draw_newflip_chart()
        
    @staticmethod
    def _rolling_mean(seq, k):
        """Simple rolling mean (window k) with warm-up (shorter windows at start)."""
        if k <= 1 or not seq:
            return list(seq)
        out = []
        s = 0.0
        from collections import deque
        dq = deque()
        for x in seq:
            x = float(x)
            dq.append(x); s += x
            if len(dq) > k:
                s -= dq.popleft()
            out.append(s / len(dq))
        return out

    def _draw_newflip_chart(self):
        c = self.nf_canvas
        c.delete("all")
        w, h = self.nf_canvas_w, self.nf_canvas_h
        c.create_rectangle(1, 1, w-1, h-1, outline="#ccc")

        # keep x-axes aligned across charts
        n = max(len(self.epsilon_history), len(self.newflip_raw_history), 1)
        if n <= 1:
            return

        # ---- cap settings ----
        CAP = 100  # clamp only for rendering

        # layout
        lp, rp = 28, 8   # left padding for y-axis labels
        xs = n - 1
        step = (w - lp - rp) / xs if xs else (w - lp - rp)

        # series
        series_raw = list(self.newflip_raw_history)
        try:
            k = int(self.nf_smooth_window.get()) if hasattr(self, 'nf_smooth_window') else 1
        except Exception:
            k = 10
        k = max(1, min(100, k))
        series_smooth = self._rolling_mean(series_raw, k) if series_raw else []

        # y-axis bounds: based on raw range (so smoothed doesn't hide spikes)
        if series_raw:
            raw_min = min(series_raw)
            raw_max = max(series_raw)
        else:
            raw_min, raw_max = -1, 1

        y_max = min(CAP, max(1, raw_max))     # cap upper bound at 100 for drawing
        y_min = min(-1, raw_min)              # lower bound natural (allow negatives)
        if y_min >= y_max:
            y_min = y_max - 1
        rng = max(1e-6, (y_max - y_min))

        # zero line
        if y_min < 0 < y_max:
            y0 = 10 + (y_max - 0) * (h - 20) / rng
            c.create_line(lp, y0, w - rp, y0, fill="#ddd")

        # y-axis ticks/labels (5 ticks)
        ticks = 5
        for ti in range(ticks):
            val = y_min + (rng * ti / (ticks - 1))
            y = 10 + (y_max - val) * (h - 20) / rng
            c.create_line(lp - 4, y, lp, y, fill="#888")
            c.create_text(lp - 6, y, text=f"{int(round(val))}", anchor="e",
                          fill="#555", font=("Arial", 8))

        # RAW line (faint)
        if series_raw:
            pts = []
            for i, val in enumerate(series_raw):
                # if epsilon history is longer (deck/tile alignment), extend last x
                x = lp + min(i, n-1) * step
                val_clamped = min(val, CAP)
                y = 10 + (y_max - val_clamped) * (h - 20) / rng
                pts += [x, y]
            if len(pts) >= 4:
                c.create_line(*pts, fill="#bbb", width=1)

        # SMOOTHED line (primary)
        if series_smooth:
            pts = []
            for i, val in enumerate(series_smooth):
                x = lp + min(i, n-1) * step
                val_clamped = min(val, CAP)
                y = 10 + (y_max - val_clamped) * (h - 20) / rng
                pts += [x, y]
            if len(pts) >= 4:
                c.create_line(*pts, fill="#36c", width=2)

        # small legend-ish hint for current k
        c.create_text(lp + 4, 12, text=f"k={k}", anchor="w", fill="#555", font=("Arial", 8))


    # ---------- Sinusoidal epsilon update (after tile reveals) ----------
    def _tick_reveal_event(self, is_tile):
        """
        Advance the shared reveal timeline by 1 step.
        - If this step is a *tile* reveal: advance ε sinusoidally and append.
        - If this step is a *deck* reveal: append current ε (no change) to keep x aligned.
        """
        import math

        self.reveal_tick += 1
        if is_tile:
            self.tile_tick += 1
            eps = 0 + 0.5 * math.sin(2.0 * math.pi * self.tile_tick / float(self.eps_period))
            # clamp to [0.00, 0.50]
            if eps < 0.0: eps = 0.00
            if eps > 0.50: eps = 0.50
            self.diff_var.set(eps)
        else:
            eps = float(self.diff_var.get())  # carry forward current ε for deck reveals

        self.epsilon_history.append(eps)
        self._draw_epsilon_chart()
        self.update_stats_panel()


    # ---------- Layout / refresh ----------
    def _build_layout(self):
        mx=max(self.board.layer_counts)
        for L,count in enumerate(self.board.layer_counts):
            tk.Label(self.board_frame,text=" " * ((mx-count)//2)).grid(row=L,column=0)
            for c in range(count):
                tid=self.board.index[(L,c)]
                btn=tk.Button(self.board_frame,text="■",width=4,height=2,command=lambda tid=tid:self.on_tile_click(tid))
                btn.grid(row=L,column=c+1,padx=2,pady=2); self.tile_buttons[tid]=btn

    def _refresh_all(self):
        for t in self.board.tiles:
            btn=self.tile_buttons[t.id]
            if t.cleared: btn.config(text=" ",state="disabled",relief="flat")
            elif t.revealed: btn.config(text=(t.letter or "?"),state="normal",relief="raised")
            else: btn.config(text="■",state="disabled",relief="ridge")
        for src,val in self.selected:
            if src=='board' and val in self.tile_buttons: self.tile_buttons[val].config(relief="sunken")

        for b in getattr(self, "stack_btns", []): b.destroy()
        self.stack_btns=[]
        for i,dc in enumerate(self.stack):
            btn=tk.Button(self.stack_frame,text=dc.letter,width=4,height=2); btn.grid(row=0,column=i,padx=2); self.stack_btns.append(btn)

        top_run,next_idx=self._stack_top_selected_run()
        for i,dc in enumerate(self.stack):
            btn=self.stack_btns[i]
            if dc.selected:
                btn.config(relief="sunken",state="normal",command=lambda i=i:self.on_stack_click(i))
            elif i==next_idx:
                btn.config(relief="raised",state="normal",command=lambda i=i:self.on_stack_click(i))
            else:
                btn.config(relief="ridge",state="disabled")

        self.deck_var.set(f"Deck remaining: {self.deck.remaining()}"); self.update_preview()

    # ---------- Reveal impact helpers ----------
    def _record_reveal_stats(self, delta_raw):
        delta_pos=max(0,int(delta_raw))
        self.reveal_events += 1
        self.total_new_words_from_flips += delta_pos
        # Track full raw Δ for plotting
        self.newflip_raw_history.append(int(delta_raw))
        # keep charts reasonably short if you want; or keep full history
        if len(self.newflip_raw_history) > 300:
            self.newflip_raw_history = self.newflip_raw_history[-300:]
        self._draw_newflip_chart()
        self.update_stats_panel()
        return delta_pos

    # ---------- Interaction ----------
    def on_tile_click(self, tid):
        if self.game_over: return
        t=self.board.tiles[tid]
        if not t.revealed or t.cleared: return
        for k,(src,val) in enumerate(self.selected):
            if src=='board' and val==tid:
                self.selected.pop(k); self.board.set_soft_clear(tid,False); self.board.recompute_visibility()
                self._refresh_all(); self.check_end_state(); return
        self.selected.append(('board',tid)); self.board.set_soft_clear(tid,True)
        self.board.recompute_visibility(); self._refresh_all(); self.check_end_state()

    def on_stack_click(self, idx):
        if self.game_over: return
        dc=self.stack[idx]
        if dc.selected:
            dc.selected=False
            for k in range(len(self.selected)-1,-1,-1):
                src,val=self.selected[k]
                if src=='stack' and val==idx: self.selected.pop(k); break
        else:
            dc.selected=True; self.selected.append(('stack',idx))
        self._refresh_all(); self.check_end_state()
        
    def draw_from_deck(self):
        if self.game_over: return
        card = self.deck.draw_one(game=self)  # <-- pass game context in
        if card is None:
            self.draw_btn.config(state="disabled")
            return
        # `card.letter` is already set by the Deck (according to the active mode)
        self.stack.append(card)
        self._refresh_all()
        if not self.deck.can_draw():
            self.draw_btn.config(state="disabled")
        self.check_end_state()

    def current_word(self):
        letters=[(self.board.tiles[v].letter or "") if s=='board' else self.stack[v].letter for s,v in self.selected]
        return "".join(letters).lower()

    def update_preview(self):
        w=self.current_word(); scr=score_word(w) if (w in LEXICON) else 0
        self.preview_var.set(f"Word: {w or '—'}  (score: {scr})")
        self.confirm_btn.config(state=("normal" if (not self.game_over) and w in LEXICON and len(w)>=2 else "disabled"))

    def clear_selection(self):
        if self.game_over: return
        for s,v in self.selected:
            if s=='board': self.board.set_soft_clear(v,False)
            else: self.stack[v].selected=False
        self.selected=[]; self.board.recompute_visibility(); self._refresh_all(); self.check_end_state()

    def backspace_last(self):
        if self.game_over or not self.selected: return
        s,v=self.selected.pop()
        if s=='board': self.board.set_soft_clear(v,False); self.board.recompute_visibility()
        else: self.stack[v].selected=False
        self._refresh_all(); self.check_end_state()

    def _kb_backspace(self,_): self.backspace_last()

    def on_confirm(self):
        if self.game_over: return
        w=self.current_word()
        if not (w in LEXICON and len(w)>=2): messagebox.showinfo("Invalid","Not a valid word."); return
        self.stats["words"]+=1; self.stats["total_len"]+=len(w); gained=score_word(w); self.stats["total_score"]+=gained

        self.score_total+=gained; self.score_var.set(f"Score: {self.score_total}")

        top_run,_=self._stack_top_selected_run()
        board_to_clear=[v for s,v in self.selected if s=='board']; self.board.clear_selected(board_to_clear)
        for _ in range(top_run): self.stack.pop()
        for dc in self.stack: dc.selected=False
        self.selected=[]; self.board.recompute_visibility(); self._refresh_all(); self.update_stats_panel(); self.check_end_state()

    # ---------- Suggestion helpers ----------
    def _build_board_pool(self, exclude_ids):
        pool={}
        for t in self.board.tiles:
            if t.revealed and (not t.cleared) and (t.id not in exclude_ids):
                ch=(t.letter or "").lower(); pool.setdefault(ch,[]).append(t.id)
        return pool

    def ui_suggest_longest_board(self):
        best=self.find_longest_word_from_board()
        if best is None: messagebox.showinfo("Suggestion","No valid completion found from the board."); return
        word,ids=best
        if messagebox.askyesno("Suggestion", f"Best word: {word.upper()}  (adds {len(ids)} board tile(s))\n\nApply this completion?"):
            for tid in ids:
                if not self.board.tiles[tid].revealed or self.board.tiles[tid].cleared: continue
                self.selected.append(('board',tid)); self.board.set_soft_clear(tid,True)
            self.board.recompute_visibility(); self._refresh_all()

    def find_longest_word_from_board(self):
        prefix=self.current_word(); sel={v for s,v in self.selected if s=='board'}
        pool=self._build_board_pool(sel)
        cands=[w for w in LEXICON if w.startswith(prefix)]
        if not cands: return None
        cands.sort(key=len,reverse=True)
        for w in cands:
            suf=w[len(prefix):]
            if not suf: return (w,[])
            avail={ch:list(ids) for ch,ids in pool.items()}; plan=[]; ok=True
            for ch in suf:
                ids=avail.get(ch)
                if not ids: ok=False; break
                plan.append(ids.pop())
            if ok: return (w,plan)
        return None

    def ui_suggest_longest_board_stack(self):
        try: max_stack=max(0,int(self.stack_spin.get()))
        except Exception: max_stack=0
        best=self.find_longest_with_stack(max_stack)
        if best is None: messagebox.showinfo("Suggestion","No valid completion using board+deck."); return
        word,actions=best
        add_board=sum(1 for a in actions if a[0]=='board'); add_stack=len(actions)-add_board
        if messagebox.askyesno("Suggestion", f"Best word: {word.upper()}  (adds {add_board} board, {add_stack} deck)"):
            for src,val in actions:
                if src=='board':
                    if not self.board.tiles[val].revealed or self.board.tiles[val].cleared: continue
                    self.selected.append(('board',val)); self.board.set_soft_clear(val,True)
                else:
                    self.stack[val].selected=True; self.selected.append(('stack',val))
            self.board.recompute_visibility(); self._refresh_all()

    def find_longest_with_stack(self, max_stack_use):
        prefix=self.current_word(); sel={v for s,v in self.selected if s=='board'}
        board_pool=self._build_board_pool(sel)
        top_run,_=self._stack_top_selected_run()
        next_idx=len(self.stack)-1-top_run
        usable=[]; i=next_idx; used=0
        while used<max_stack_use and i>=0:
            if self.stack[i].selected: break
            usable.append((self.stack[i].letter.lower(),i)); used+=1; i-=1

        cands=[w for w in LEXICON if w.startswith(prefix)]
        if not cands: return None
        cands.sort(key=len,reverse=True)

        def plan_suffix(suf,board_av,stack_seq,wpos,acts):
            if not suf: return True
            ch=suf[0]; ids=board_av.get(ch)
            if ids:
                tid=ids.pop(); acts.append(('board',tid))
                if plan_suffix(suf[1:],board_av,stack_seq,wpos,acts): return True
                acts.pop(); ids.append(tid)
            if wpos<len(stack_seq) and stack_seq[wpos][0]==ch:
                _,sidx=stack_seq[wpos]; acts.append(('stack',sidx))
                if plan_suffix(suf[1:],board_av,stack_seq,wpos+1,acts): return True
                acts.pop()
            return False

        for w in cands:
            suf=w[len(prefix):]
            if not suf: return (w,[])
            board_av={ch:list(ids) for ch,ids in board_pool.items()}; acts=[]
            if plan_suffix(suf,board_av,usable,0,acts): return (w,acts)
        return None

    # ---------- Lexicon precompute / counts ----------
    def _precompute_lexicon_counts(self):
        self._lex_words=list(LEXICON)
        self._lex_words_sorted=sorted(self._lex_words,key=len,reverse=True)
        self._lex_counts=[]
        for w in self._lex_words:
            d={}
            for ch in w: d[ch]=d.get(ch,0)+1
            self._lex_counts.append(d)
        idx_map={w:i for i,w in enumerate(self._lex_words)}
        self._lex_counts_sorted=[self._lex_counts[idx_map[w]] for w in self._lex_words_sorted]

    @staticmethod
    def _add_counts(a,b):
        out=dict(a)
        for k,v in b.items(): out[k]=out.get(k,0)+v
        return out

    @staticmethod
    def _counts_fit(need,have):
        for ch,n in need.items():
            if n>have.get(ch,0): return False
        return True

    def _next_round_counts(self):
        # revealed, unselected board
        sel={v for s,v in self.selected if s=='board'}
        board_counts={}
        for t in self.board.tiles:
            if t.revealed and (not t.cleared) and (t.id not in sel):
                ch=(t.letter or "").lower(); board_counts[ch]=board_counts.get(ch,0)+1
        # contiguous deck-stack tail (top→down), ignoring already-selected top block
        top_run,_=self._stack_top_selected_run()
        next_idx=len(self.stack)-1-top_run
        stack_seq=[]; i=next_idx
        while i>=0 and not self.stack[i].selected:
            stack_seq.append(self.stack[i].letter.lower()); i-=1
        stack_counts={}
        for ch in stack_seq: stack_counts[ch]=stack_counts.get(ch,0)+1
        return board_counts, stack_counts, stack_seq

    def _can_form_with_resources(self, word, board_counts, stack_seq):
        need=dict(board_counts); wpos=0
        for ch in word:
            if need.get(ch,0)>0: need[ch]-=1
            elif wpos<len(stack_seq) and stack_seq[wpos]==ch: wpos+=1
            else: return False
        return True

    def _exact_count_next_round(self, base_counts, stack_seq, extra_letter=None):
        base_plus=dict(base_counts)
        if extra_letter: base_plus[extra_letter]=base_plus.get(extra_letter,0)+1
        cands=[w for w,need in zip(self._lex_words,self._lex_counts) if self._counts_fit(need,base_plus)]
        board_counts,_,_ = self._next_round_counts()
        if extra_letter:
            board_counts=dict(board_counts); board_counts[extra_letter]=board_counts.get(extra_letter,0)+1
        total=0
        for w in cands:
            if self._can_form_with_resources(w,board_counts,list(stack_seq)): total+=1
        return total

    def _exact_longest_next_round(self, base_counts, stack_seq, extra_letter=None):
        base_plus=dict(base_counts)
        if extra_letter: base_plus[extra_letter]=base_plus.get(extra_letter,0)+1
        board_counts,_,_ = self._next_round_counts()
        if extra_letter:
            board_counts=dict(board_counts); board_counts[extra_letter]=board_counts.get(extra_letter,0)+1
        for w,need in zip(self._lex_words_sorted,self._lex_counts_sorted):
            if not self._counts_fit(need,base_plus): continue
            if self._can_form_with_resources(w,board_counts,list(stack_seq)): return w
        return None

    # ---------- Difficulty-controlled pickers ----------
    def _pick_letter_by_param(self, base_counts, stack_seq, tag, tile=None):
        """
        Evaluate Δ(new next-round words) for all A–Z, then pick the letter whose Δ
        is closest to target_delta, where:
          target_delta = min_delta * diff + max_delta * (1 - diff)
        diff=0 → aim for max_delta (easier), diff=1 → aim for min_delta (harder)
        Tie-breakers: if diff>=0.5 prefer shorter longest-word; else prefer longer.
        """
        before = self._exact_count_next_round(base_counts, stack_seq, extra_letter=None)
        evals = []  # (ch, delta, after, longest_len, longest_word)
        for ch in "ABCDEFGHIJKLMNOPQRSTUVWXYZ":
            after = self._exact_count_next_round(base_counts, stack_seq, extra_letter=ch.lower())
            longest = self._exact_longest_next_round(base_counts, stack_seq, extra_letter=ch.lower())
            delta = after - before
            evals.append((ch, delta, after, len(longest) if longest else 0, longest or "—"))
        if not evals:
            ch = sample_letter()
            self.log(f"[{tag}] Fallback random '{ch}'")
            return ch, 0, 0, "—"

        min_delta = min(d for _, d, _, _, _ in evals)
        max_delta = max(d for _, d, _, _, _ in evals)
        diff = float(self.diff_var.get())
        target = (min_delta * diff) + (max_delta * (1.0 - diff))

        if diff >= 0.5:
            key = lambda e: (abs(e[1]-target), e[3], BASE_LETTER_WEIGHTS.get(e[0],0))
        else:
            key = lambda e: (abs(e[1]-target), -e[3], -BASE_LETTER_WEIGHTS.get(e[0],0))

        ch, delta, after, Llen, Lword = min(evals, key=key)
        return ch, delta, after, Lword

    def pick_letter_for_reveal(self, tile):
        board_counts, stack_counts, stack_seq = self._next_round_counts()
        base_counts=self._add_counts(board_counts, stack_counts)

        ch, dr, after, longest = self._pick_letter_by_param(base_counts, stack_seq, tag="TILE", tile=tile)
        dpos=self._record_reveal_stats(dr)
        self.log(f"[TILE] Reveal (L{tile.layer},c{tile.col}) -> '{ch}': "
                 f"+{dpos} words (Δ={dr:+d}; next-round total {after}); "
                 f"longest: {str(longest).upper()}  | ε={self.diff_var.get():.2f}")
        # IMPORTANT: advance ε AFTER the tile reveal so the next reveal uses updated difficulty
        self._tick_reveal_event(is_tile=True)
        return ch

    def pick_letter_for_deck_reveal(self):
        board_counts, stack_counts, stack_seq = self._next_round_counts()
        base_counts=self._add_counts(board_counts, stack_counts)

        ch, dr, after, longest = self._pick_letter_by_param(base_counts, stack_seq, tag="DECK")
        dpos=self._record_reveal_stats(dr)
        self.log(f"[DECK] Flip -> '{ch}': +{dpos} words (Δ={dr:+d}; next-round total {after}); "
                 f"longest: {str(longest).upper()}  | ε={self.diff_var.get():.2f}")
        self._tick_reveal_event(is_tile=False)
        return ch

    # ---------- End-state detection ----------
    def check_end_state(self):
        if self.game_over: return
        if all(t.cleared for t in self.board.tiles):
            self.log(f"Game score {self.score_total} → {self._classify_game_score(self.score_total).upper()}"); self.end_game("WIN"); return
        board_counts={}
        for t in self.board.tiles:
            if t.revealed and (not t.cleared):
                ch=(t.letter or "").lower(); board_counts[ch]=board_counts.get(ch,0)+1
        stack_seq=[self.stack[i].letter.lower() for i in range(len(self.stack)-1,-1,-1)]
        stack_counts={}; [stack_counts.__setitem__(ch, stack_counts.get(ch,0)+1) for ch in stack_seq]
        base_counts=self._add_counts(board_counts,stack_counts)
        cands=[w for w,need in zip(self._lex_words,self._lex_counts) if self._counts_fit(need,base_counts)]
        possible=any(self._can_form_with_resources(w,dict(board_counts),list(stack_seq)) for w in cands)
        if possible: return
        if self.deck.can_draw():
            self.game_status.set("Game: No words — draw from deck to continue."); self.log("No words available now, but deck still has cards. Draw to continue."); return
        self.log(f"Game score {self.score_total} → {self._classify_game_score(self.score_total).upper()}"); self.end_game("LOSS")

    # ---------- Bot ----------
    def run_bot_10(self):
        self.run_bot_btn.config(state="disabled"); self.new_game_btn.config(state="disabled")
        self.log("== BOT START (10 games) =="); self.root.update_idletasks(); self.root.update()
        for g in range(10):
            self.log(f"-- Bot: Game {g+1}/10 --"); self.start_new_board(); self.root.update_idletasks(); self.root.update()
            safety=5000
            while not self.game_over and safety>0:
                safety-=1
                if self.selected: self.clear_selection(); self.root.update_idletasks(); self.root.update()
                if self.game_over: break
                best=self.find_longest_with_stack(len(self.stack))
                if best is None:
                    while best is None and self.deck.can_draw() and not self.game_over:
                        self.draw_from_deck(); self.root.update_idletasks(); self.root.update()
                        best=self.find_longest_with_stack(len(self.stack))
                    if best is None:
                        self.check_end_state(); self.root.update_idletasks(); self.root.update(); break
                word,actions=best
                for s,v in actions:
                    if s=='board':
                        if not self.board.tiles[v].revealed or self.board.tiles[v].cleared: continue
                        self.selected.append(('board',v)); self.board.set_soft_clear(v,True)
                    else:
                        self.stack[v].selected=True; self.selected.append(('stack',v))
                self.board.recompute_visibility(); self._refresh_all(); self.root.update_idletasks(); self.root.update()
                wstr=self.current_word()
                if wstr in LEXICON and len(wstr)>=2: self.on_confirm(); self.root.update_idletasks(); self.root.update()
                else: self.clear_selection(); self.check_end_state(); self.root.update_idletasks(); self.root.update()
            if not self.game_over: self.log("-- Bot safety stop; counting as LOSS --"); self.end_game("LOSS")
            self.root.update_idletasks(); self.root.update()
        self.log("== BOT END =="); self.run_bot_btn.config(state="normal")
        if self.game_over: self.new_game_btn.config(state="normal")

# ----------------------------
# Run
# ----------------------------
#if __name__=="__main__":
#    random.seed(); root=tk.Tk(); app=GameUI(root); root.mainloop()

In [2]:
# ----------------------------
# Puzzle generation (standalone)
# ----------------------------
import math, random, string
from collections import Counter, defaultdict

class PuzzleGenerator:
    """
    Generates full-board letter assignments given a DepthBoard and difficulty D in (0,1).

    Intuition
    ---------
    1) Build a connectivity matrix C[i,j]∈[0,1] capturing how likely two tiles i,j
       are to be usable together (first row = 1.0; deeper rows taper; direct/indirect
       blockers reduce).
    2) Build letter/bigram statistics from the lexicon to estimate 'wordability'
       when two letters co-occur on connected tiles.
    3) Place letters row-by-row. Each candidate letter for a tile scores against the
       letters already placed on connected tiles. Low difficulty boosts high-scoring
       letters (friendlier board). High difficulty steers away from them.

    Usage
    -----
        gen = PuzzleGenerator(LEXICON, BASE_LETTER_WEIGHTS)
        letters_by_tid = gen.generate_puzzle(board, difficulty=0.65, seed=None)

        # Apply to your board (before the first reveal happens):
        for t in board.tiles:
            t.letter = letters_by_tid[t.id]  # uppercase letters
    """

    def __init__(self, lexicon, base_letter_weights=None, max_word_len=6):
        self.lexicon = [w for w in lexicon if 2 <= len(w) <= max_word_len and w.isalpha()]
        self.base_letter_weights = dict(base_letter_weights or {})
        self.max_word_len = max_word_len

        # Precompute letter & bigram stats from lexicon
        self.letter_freq, self.bigram_freq = self._build_lex_stats(self.lexicon)
        self.vowels = set("aeiou")
        # Smoothing for bigram PMI-like score
        self._bg_total = sum(self.bigram_freq.values()) + 1.0
        self._lt_total = sum(self.letter_freq.values()) + 1.0

    # ----------------------------
    # Public API
    # ----------------------------
    def generate_puzzle(self, board, difficulty, seed=None):
        """
        Return mapping {tile_id: 'A'} for the entire board, tuned by difficulty ∈ (0,1).
        """
        assert 0.0 < difficulty < 1.0, "difficulty must be strictly between 0 and 1"

        if seed is not None:
            random.seed(seed)

        # 1) connectivity
        C = self._compute_connectivity(board)  # NxN matrix in flat dict {(i,j): weight}

        # 2) placement order (row by row, left->right)
        rows = self._rows_from_board(board)
        placement_order = [tid for row in rows for tid in row]

        # 3) choose letter per tile
        placed = {}  # tid -> uppercase letter
        for tid in placement_order:
            letter = self._pick_letter_for_tile(board, tid, placed, C, difficulty)
            placed[tid] = letter

        return placed

    # ----------------------------
    # Connectivity
    # ----------------------------
    def _compute_connectivity(self, board):
        """
        Build a symmetric connectivity matrix C[i,j] in [0,1].
        Heuristics:
          - Same-row baseline decays with layer depth (row 0 = 1.0).
          - Cross-row connectivity = geometric mean of row baselines.
          - If one directly blocks the other (NEW: no effect)
          - If they share blockers, moderately LIFT.
        """
        N = len(board.tiles)
        C = {}
        # Per-row baseline: deeper rows less likely to be concurrently available
        max_layer = max(t.layer for t in board.tiles) if N else 0
        row_baseline = {}
        for L in range(max_layer+1):
            if L == 0:
                row_baseline[L] = 1.00
            else:
                # Smooth decay; tweakable
                row_baseline[L] = max(0.15, 0.75 * (1.0 - (L / (max_layer + 0.0001))**0.9))

        # Precompute blocker relations
        blocked_by = {t.id: set(t.blocked_by) for t in board.tiles}
        blocks = defaultdict(set)
        for i, t in enumerate(board.tiles):
            for b in t.blocked_by:
                blocks[b].add(t.id)

        # Compute connectivity
        for i in range(N):
            ti = board.tiles[i]
            for j in range(N):
                if i == j:
                    C[(i, j)] = 1.0
                    continue
                tj = board.tiles[j]
                # Baseline by rows
                base = math.sqrt(row_baseline[ti.layer] * row_baseline[tj.layer])

                # Structural penalties
                pen = 1.0
                # direct blocker relation
                if (i in blocked_by[j]) or (j in blocked_by[i]):
                    pen *= 1   # no nothing
                else:
                    # shared blockers increase chance of sequencing conflicts
                    shared = blocked_by[i].intersection(blocked_by[j])
                    if shared:
                        pen *= 2 # large connectivity bonus
                    # if i and j block the same third tiles (lateral conflict), smaller penalty
                    lateral = blocks[i].intersection(blocks[j])
                    if lateral:
                        pen *= 1.1 # small connectivity bonus

                # Light geometric distance penalty across columns in same row (spreads clusters)
                if ti.layer == tj.layer:
                    dist = abs(ti.col - tj.col)
                    if dist >= 2:
                        pen *= 0.92
                    if dist >= 4:
                        pen *= 0.85

                C[(i, j)] = max(0.0, min(1.0, base * pen))
        return C

    def _rows_from_board(self, board):
        """Return [[tid,...], ...] row-wise left->right."""
        per_row = defaultdict(list)
        for t in board.tiles:
            per_row[t.layer].append(t.id)
        rows = []
        for L in range(len(board.layer_counts)):
            row = sorted(per_row[L], key=lambda tid: board.tiles[tid].col)
            rows.append(row)
        return rows

    # ----------------------------
    # Lexicon stats & scoring
    # ----------------------------
    def _build_lex_stats(self, lexicon):
        lt = Counter()
        bg = Counter()
        for w in lexicon:
            w = w.strip().lower()
            if not w.isalpha():
                continue
            for ch in w:
                lt[ch] += 1
            for a, b in zip(w, w[1:]):
                bg[a + b] += 1
        return lt, bg

    def _pmi_like(self, a, b):
        """
        Positive association score for bigram a->b using smoothed PMI-ish value.
        Returns roughly in [-?, +?]; higher means more helpful adjacency.
        """
        a = a.lower(); b = b.lower()
        pab = (self.bigram_freq.get(a + b, 0) + 1.0) / self._bg_total
        pa = (self.letter_freq.get(a, 0) + 1.0) / self._lt_total
        pb = (self.letter_freq.get(b, 0) + 1.0) / self._lt_total
        return math.log(pab / (pa * pb))

    def _is_vowel(self, ch):
        return ch.lower() in self.vowels

    def _letter_base_weight(self, ch):
        """
        Prior for letter sampling. Defaults to BASE_LETTER_WEIGHTS (if provided),
        else uses lexicon frequency.
        """
        ch = ch.upper()
        if self.base_letter_weights:
            return float(self.base_letter_weights.get(ch, 1.0))
        # fallback to lexicon frequency prior
        return 1.0 + self.letter_freq.get(ch.lower(), 0)

    def _wordability_gain(self, ch_new, placed_letters, conn_weights):
        """
        Score how much placing `ch_new` (char) at the current tile helps/hurts
        forming words with already-placed neighbor letters.
        placed_letters: iterable of uppercase letters already placed on other tiles
        conn_weights:   iterable of corresponding connectivity weights in [0,1]
        """
        # Bigram gain (both directions) + mild vowel balance encouragement
        gain = 0.0
        for ch_old, w in zip(placed_letters, conn_weights):
            # bigram both ways, scaled by connectivity weight
            gain += w * (self._pmi_like(ch_old, ch_new) + self._pmi_like(ch_new, ch_old)) * 0.5

            # small nudge: diverse vowels/consonants help (don’t overdo)
            if self._is_vowel(ch_old) != self._is_vowel(ch_new):
                gain += 0.25 * w

        return gain

    # ----------------------------
    # Letter placement
    # ----------------------------
    def _pick_letter_for_tile(self, board, tid, placed, C, difficulty):
        """
        Compute a score for each A-Z and sample with a temperature that flips preference
        as difficulty rises:
          - Easy (D≈0): favor high 'wordability_gain'
          - Hard (D≈1): favor low/negative 'wordability_gain'
        """
        # Gather neighbors that are already placed
        nbr_ids, nbr_letters, nbr_conn = [], [], []
        for j in range(len(board.tiles)):
            if j == tid:
                continue
            if j in placed:
                nbr_ids.append(j)
                nbr_letters.append(placed[j])
                nbr_conn.append(C[(tid, j)])

        # Temperature and polarity schedule:
        #   polarity = +1 at D=0 (seek friendly boards)
        #   polarity = -1 at D=1 (avoid friendly collocations)
        #   temp     increases with D to add randomness on hard boards
        polarity = 1.0 - 2.0 * difficulty           # ∈ (+1 .. -1)
        temperature = 0.25 + 2.75 * difficulty      # ∈ (~0.25 .. 3.0)

        # Evaluate candidates
        cand_scores = []
        alphabet = string.ascii_uppercase
        for ch in alphabet:
            base = math.log(self._letter_base_weight(ch) + 1e-9)
            gain = self._wordability_gain(ch, nbr_letters, nbr_conn)
            score = base + polarity * gain
            cand_scores.append((ch, score))

        # Softmax sampling with temperature
        # (On hard settings, higher temp flattens; polarity flips preferences.)
        max_s = max(s for _, s in cand_scores)
        exps = []
        for ch, s in cand_scores:
            exps.append((ch, math.exp((s - max_s) / max(1e-6, temperature))))
        Z = sum(v for _, v in exps)
        r = random.random() * Z
        acc = 0.0
        for ch, v in exps:
            acc += v
            if acc >= r:
                return ch
        return exps[-1][0]  # fallback, should never hit

In [3]:
# puzzle_io.py
import json

def save_puzzle_json(path, difficulty, board, letters_by_tid, seed=None, extra_meta=None):
    """
    Persist a generated puzzle to JSON so it can be replayed later.
    """
    meta = dict(extra_meta or {})
    payload = {
        "version": 1,
        "difficulty": float(difficulty),
        "seed": seed,
        "layer_counts": list(board.layer_counts),
        # store letters by tile id as strings
        "letters_by_tid": {int(tid): str(ch) for tid, ch in letters_by_tid.items()},
        "meta": meta,
    }
    with open(path, "w", encoding="utf-8") as f:
        json.dump(payload, f, ensure_ascii=False, indent=2)


def load_puzzle_json(path):
    """
    Return (payload_dict). You will usually read `layer_counts` and `letters_by_tid`.
    """
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)


def apply_puzzle_letters(board, letters_by_tid):
    """
    Assign pre-generated letters to the board tiles by id.
    Call this BEFORE the board starts revealing (i.e., right after DepthBoard() is built).
    """
    for t in board.tiles:
        # only set if provided; leaves others to your existing sampler
        ch = letters_by_tid.get(str(t.id), letters_by_tid.get(int(t.id)))
        if ch:
            t.letter = ch.upper()


In [4]:
# make_puzzle_easy.py
import random
import numpy as np
#from wordscapes_depth import DepthBoard, LEXICON, BASE_LETTER_WEIGHTS
#from puzzle_io import save_puzzle_json
#from puzzle_generator import PuzzleGenerator  # the class I sent earlier

def main():
    # build the same topology you use in-game
    board = DepthBoard(layer_counts=(6, 8, 10, 12))

    # generate at difficulty 0.10 (easy)
    seed = 12345  # optional, so you can reproduce the exact board
    random.seed(seed)

    gen = PuzzleGenerator(LEXICON, BASE_LETTER_WEIGHTS)

    for D in np.arange(1,100,1):
        letters_by_tid = gen.generate_puzzle(board, difficulty=D/100, seed=seed)
        # save it to disk
        save_puzzle_json(
            path=f"puzzles/{D}.json",
            difficulty=D/100,
            board=board,
            letters_by_tid=letters_by_tid,
            seed=seed,
            extra_meta={"note": ""}
        )

#if __name__ == "__main__":
#    main()

C:\Users\Brian\Anaconda3\envs\tf_2\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Brian\Anaconda3\envs\tf_2\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\Brian\Anaconda3\envs\tf_2\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [5]:
def new_game_from_file(self, path):
    payload = load_puzzle_json(path)
    self.board = DepthBoard(layer_counts=tuple(payload["layer_counts"]))
    apply_puzzle_letters(self.board, payload["letters_by_tid"])
    self.deck = Deck(size=10)
    # continue exactly as start_new_board() does…
    self.stack = []; self.selected = []
    self.score_total = 0; self.score_var.set("Score: 0")
    self.game_status.set(f"Game: In progress, {path}"); self.game_over = False
    self.draw_btn.config(state="normal"); self.new_game_btn.config(state="disabled")
    for w in list(self.board_frame.grid_slaves()): w.destroy()
    self.tile_buttons = {}; self._build_layout()
    self.board.recompute_visibility(); self._refresh_all()
    self.deck_var.set(f"Deck remaining: {self.deck.remaining()}")
    self.update_stats_panel()
    self._draw_epsilon_chart(); self._draw_newflip_chart()
    
GameUI.new_game_from_file = new_game_from_file

In [6]:
# ----------------------------
# Run
# ----------------------------
if __name__=="__main__":
    random.seed(); root=tk.Tk(); app=GameUI(root); root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Brian\Anaconda3\envs\tf_2\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "C:\Users\Brian\AppData\Local\Temp/ipykernel_19268/4207680446.py", line 1103, in run_bot_10
    if wstr in LEXICON and len(wstr)>=2: self.on_confirm(); self.root.update_idletasks(); self.root.update()
  File "C:\Users\Brian\AppData\Local\Temp/ipykernel_19268/4207680446.py", line 840, in on_confirm
    self.selected=[]; self.board.recompute_visibility(); self._refresh_all(); self.update_stats_panel(); self.check_end_state()
  File "C:\Users\Brian\AppData\Local\Temp/ipykernel_19268/4207680446.py", line 729, in _refresh_all
    if t.cleared: btn.config(text=" ",state="disabled",relief="flat")
  File "C:\Users\Brian\Anaconda3\envs\tf_2\lib\tkinter\__init__.py", line 1485, in configure
    return self._configure('configure', cnf, kw)
  File "C:\Users\Brian\Anaconda3\envs\tf_2\lib\tkinter\__init__.py", 